In [51]:
import pandas as pd
import numpy as np
%matplotlib inline
import lightgbm as lgb
from tqdm import tqdm_notebook
import warnings
warnings.simplefilter("ignore")

# Começaremos às 20:05
## Quer que eu te ensine como eu faço projetos de Data Science? Acesse CursoDeDataScience.com.br

- Alguém com conhecimento de epidemiologia aqui? Eu não entendo nada.
- Muitas questões serão esclarecidas na prática:
    - poucos dados
    - dados com problemas "graves"
    - como criar um training/validation set a partir dos dados brutos
    - exemplos de baselines fortes (nada de ficar enganando só pra dizer que o modelo funcionou)
    - a "arte" que é Machine Learning na prática (se você parar pra pensar, estamos ferindo 7585747438 suposições de modelos, técnicas e etc) e a coisa funciona


[https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset]

[https://www.kaggle.com/fernandol/countries-of-the-world]

[https://www.kaggle.com/max-mind/world-cities-database]

[https://www.kaggle.com/andradaolteanu/iso-country-codes-global]

[https://www.kaggle.com/folaraz/world-countries-and-continents-details]

https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6

In [2]:
covid = pd.read_csv("covid_19_data.csv")

In [3]:
covid['ObservationDate'] = pd.to_datetime(covid['ObservationDate'], format='%m/%d/%Y')

In [4]:
covid2 = covid.drop_duplicates(['ObservationDate', 'Country/Region'])


#.drop_duplicates(subset= ['Country/Region'], keep='first')

In [5]:
covid2 = covid2[covid2['Confirmed'] > 0]

In [6]:
covid2['Country/Region'].value_counts()

Japan                   33
Taiwan                  33
Mainland China          33
South Korea             33
Thailand                33
US                      33
Macau                   33
Vietnam                 32
Singapore               32
Hong Kong               32
France                  31
Australia               30
Nepal                   30
Malaysia                30
Canada                  29
Cambodia                28
Sri Lanka               28
Germany                 27
United Arab Emirates    26
Finland                 26
Philippines             25
India                   25
Italy                   24
Russia                  24
Sweden                  24
UK                      24
Spain                   23
Belgium                 20
Others                  17
Egypt                   10
Iran                     5
Lebanon                  3
Israel                   3
Ivory Coast              1
Name: Country/Region, dtype: int64

In [7]:
covid3 = covid2.drop_duplicates("Country/Region")
covid3 = covid3[covid3["Country/Region"] != "Others"]
covid3 = covid3.reset_index(drop=True)
covid3

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,21,2020-01-22,Macau,Macau,1/22/2020 17:00,1.0,0.0,0.0
2,29,2020-01-22,Taiwan,Taiwan,1/22/2020 17:00,1.0,0.0,0.0
3,32,2020-01-22,Washington,US,1/22/2020 17:00,1.0,0.0,0.0
4,36,2020-01-22,NaN,Japan,1/22/2020 17:00,2.0,0.0,0.0
5,37,2020-01-22,NaN,Thailand,1/22/2020 17:00,2.0,0.0,0.0
6,38,2020-01-22,NaN,South Korea,1/22/2020 17:00,1.0,0.0,0.0
7,51,2020-01-23,Hong Kong,Hong Kong,1/23/20 17:00,2.0,0.0,0.0
8,77,2020-01-23,NaN,Singapore,1/23/20 17:00,1.0,0.0,0.0
9,80,2020-01-23,NaN,Vietnam,1/23/20 17:00,2.0,0.0,0.0


# Qual o problema com esses dados? W
- Poucos dados, mudando constantemente, mas "previsível"

In [8]:
countries = pd.read_csv("countries and continents.csv")
countries_loc = pd.read_csv("Countries Longitude and Latitude.csv", index_col=0)


In [9]:
countries.head()

,name,official_name_en,official_name_fr,ISO3166-1-Alpha-2,ISO3166-1-Alpha-3,M49,ITU,MARC,WMO,DS,...,ISO4217-currency_minor_unit,ISO4217-currency_name,ISO4217-currency_numeric_code,is_independent,Capital,Continent,TLD,Languages,Geoname ID,EDGAR
0,NaN,Channel Islands,Îles Anglo-Normandes,NaN,NaN,830,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Sark,Sercq,NaN,NaN,680,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,Afghanistan,Afghanistan,AF,AFG,4,AFG,af,AF,AFG,...,2.0,Afghani,971.0,Yes,Kabul,AS,.af,"fa-AF,ps,uz-AF,tk",1149361.0,B2
3,Albania,Albania,Albanie,AL,ALB,8,ALB,aa,AB,AL,...,2.0,Lek,8.0,Yes,Tirana,EU,.al,"sq,el",783754.0,B3
4,Algeria,Algeria,Algérie,DZ,DZA,12,ALG,ae,AL,DZ,...,2.0,Algerian Dinar,12.0,Yes,Algiers,AF,.dz,ar-DZ,2589581.0,B4


In [10]:
countries_loc.head()

,longitude,latitude,name
0,33.791638,-84.389488,NaN
1,33.791638,-84.389488,NaN
2,33.939110,67.709953,Afghanistan
3,41.153332,20.168331,Albania
4,28.033886,1.659626,Algeria


In [11]:
countries_loc2 = countries_loc.rename(columns={"name": "Country/Region"})
countries_loc2 = countries_loc2.dropna()

In [12]:
covid3.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,21,2020-01-22,Macau,Macau,1/22/2020 17:00,1.0,0.0,0.0
2,29,2020-01-22,Taiwan,Taiwan,1/22/2020 17:00,1.0,0.0,0.0
3,32,2020-01-22,Washington,US,1/22/2020 17:00,1.0,0.0,0.0
4,36,2020-01-22,NaN,Japan,1/22/2020 17:00,2.0,0.0,0.0


In [13]:
countries_loc2[countries_loc2['Country/Region'].str.contains("Iv").fillna(False)]

,longitude,latitude,Country/Region
61,7.539989,-5.54708,Côte d’Ivoire


In [14]:
countries_loc2.loc[countries_loc2['Country/Region'] == 'China', 'Country/Region'] = 'Mainland China'
countries_loc2.loc[countries_loc2['Country/Region'] == 'Côte d’Ivoire', 'Country/Region'] = 'Ivory Coast' 

In [15]:
countries_loc2

,longitude,latitude,Country/Region
2,33.939110,67.709953,Afghanistan
3,41.153332,20.168331,Albania
4,28.033886,1.659626,Algeria
5,-14.270972,-170.132217,American Samoa
6,42.506285,1.521801,Andorra
...,...,...,...
246,24.215527,-12.885834,Western Sahara
247,15.552727,48.516388,Yemen
248,-13.133897,27.849332,Zambia
249,-19.015438,29.154857,Zimbabwe


In [16]:
covid4 = covid3.merge(countries_loc2, how='left')
covid4

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,longitude,latitude
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,35.861660,104.195397
1,21,2020-01-22,Macau,Macau,1/22/2020 17:00,1.0,0.0,0.0,22.198745,113.543873
2,29,2020-01-22,Taiwan,Taiwan,1/22/2020 17:00,1.0,0.0,0.0,23.697810,120.960515
3,32,2020-01-22,Washington,US,1/22/2020 17:00,1.0,0.0,0.0,40.760537,-73.978890
4,36,2020-01-22,NaN,Japan,1/22/2020 17:00,2.0,0.0,0.0,36.204824,138.252924
5,37,2020-01-22,NaN,Thailand,1/22/2020 17:00,2.0,0.0,0.0,15.870032,100.992541
6,38,2020-01-22,NaN,South Korea,1/22/2020 17:00,1.0,0.0,0.0,35.907757,127.766922
7,51,2020-01-23,Hong Kong,Hong Kong,1/23/20 17:00,2.0,0.0,0.0,22.396428,114.109497
8,77,2020-01-23,NaN,Singapore,1/23/20 17:00,1.0,0.0,0.0,1.352083,103.819836
9,80,2020-01-23,NaN,Vietnam,1/23/20 17:00,2.0,0.0,0.0,14.058324,108.277199


In [18]:
from geopy.distance import great_circle
def dist_to_confirmed(lon, lat, confirmed_loc):
    dists = []
    for lon2, lat2, country in confirmed_loc.values:
        d = great_circle((lon, lat), (lon2, lat2))
        dists.append(d.kilometers)
    return np.array(dists)

def calc_confirmed_radius(lon, lat, country, confirmed_loc):
    res = {"Country/Region": country}
    
    dists = dist_to_confirmed(lon, lat, confirmed_loc)
    
    res['avg_dist_to_confirmed'] = np.mean(dists)
    res['min_dist_to_confirmed'] = np.min(dists)
    res['confirmed_1k'] = (dists < 1000).sum()
    res['confirmed_2k'] = (dists < 2000).sum()
    res['confirmed_3k'] = (dists < 3000).sum()
    
    return res
    
    

#a = dist_to_confirmed(33.939110, 67.709953, confirmed_loc)

In [19]:
countries_loc2.head()

,longitude,latitude,Country/Region
2,33.939110,67.709953,Afghanistan
3,41.153332,20.168331,Albania
4,28.033886,1.659626,Algeria
5,-14.270972,-170.132217,American Samoa
6,42.506285,1.521801,Andorra


# Que fontes de dados, features devem ser relevantes?

In [54]:
#https://en.wikipedia.org/wiki/Severe_acute_respiratory_syndrome
"""Country or region	Cases	Deaths	Fatality (%)
 China[a]	5,327	349	6.6
 Hong Kong	1,755	299	17.0
 Taiwan[b]	346	73 [23] [24]	21.1
 Canada	251	43	17.1
 Singapore	238	33	13.9
 Vietnam	63	5	7.9
 United States	27	0	0
 Philippines	14	2	14.3
 Thailand	9	2	22.2
 Germany	9	0	0
 Mongolia	9	0	0
 France	7	1	14.3
 Australia	6	0	0
 Malaysia	5	2	40.0
 Sweden	5	0	0
 United Kingdom	4	0	0
 Italy	4	0	0
 India	3	0	0
 South Korea	3	0	0
 Indonesia	2	0	0
 South Africa	1	1	100.0
 Kuwait	1	0	0
 Ireland	1	0	0
 Macau	1	0	0
 New Zealand	1	0	0
 Romania	1	0	0
 Russia	1	0	0
 Spain	1	0	0
  Switzerland	1	0	0
Total excluding China	2,769	454	16.4
Total (29 territories)	8,096	774	9.6"""
sars_df = pd.read_clipboard()

In [23]:
sars_df2 = sars_df.iloc[:-2]
sars_df2['Country or region'] = sars_df2['Country or region'].str.strip()
sars_df2.loc[sars_df2['Country or region'] == 'China[a]', 'Country or region'] = 'Mainland China' 
sars_df2.loc[sars_df2['Country or region'] == 'Taiwan[b]', 'Country or region'] = 'Taiwan'
sars_df2.loc[sars_df2['Country or region'] == 'United States', 'Country or region'] = 'US'
sars_df2.loc[sars_df2['Country or region'] == 'United Kingdom', 'Country or region'] = 'UK'
sars_df2['sars'] = 1
sars_df2 = sars_df2.rename(columns={'Country or region': "Country/Region"})
sars_df2

,Country/Region,Cases,Deaths,Fatality (%),sars
0,Mainland China,"5,327",349,6.6,1
1,Hong Kong,"1,755",299,17.0,1
2,Taiwan,346,73 [23] [24],21.1,1
3,Canada,251,43,17.1,1
4,Singapore,238,33,13.9,1
5,Vietnam,63,5,7.9,1
6,US,27,0,0.0,1
7,Philippines,14,2,14.3,1
8,Thailand,9,2,22.2,1
9,Germany,9,0,0.0,1


In [24]:
countries_loc3 = countries_loc2.merge(sars_df2, how='left', on='Country/Region')
countries_loc3

,longitude,latitude,Country/Region,Cases,Deaths,Fatality (%),sars
0,33.939110,67.709953,Afghanistan,NaN,NaN,NaN,NaN
1,41.153332,20.168331,Albania,NaN,NaN,NaN,NaN
2,28.033886,1.659626,Algeria,NaN,NaN,NaN,NaN
3,-14.270972,-170.132217,American Samoa,NaN,NaN,NaN,NaN
4,42.506285,1.521801,Andorra,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
244,24.215527,-12.885834,Western Sahara,NaN,NaN,NaN,NaN
245,15.552727,48.516388,Yemen,NaN,NaN,NaN,NaN
246,-13.133897,27.849332,Zambia,NaN,NaN,NaN,NaN
247,-19.015438,29.154857,Zimbabwe,NaN,NaN,NaN,NaN


In [25]:
sars_df2.shape[0],' ', (countries_loc3['sars'].notnull()).sum()

(29, ' ', 29)

In [26]:
sars_df2.sort_values("Country/Region").head()

,Country/Region,Cases,Deaths,Fatality (%),sars
12,Australia,6,0,0.0,1
3,Canada,251,43,17.1,1
11,France,7,1,14.3,1
9,Germany,9,0,0.0,1
1,Hong Kong,"1,755",299,17.0,1


In [27]:
countries_loc3[countries_loc3['sars'].notnull()].sort_values("Country/Region").head()

,longitude,latitude,Country/Region,Cases,Deaths,Fatality (%),sars
12,-25.274398,133.775136,Australia,6,0,0.0,1.0
38,56.130366,-106.346771,Canada,251,43,17.1,1.0
75,46.227638,2.213749,France,7,1,14.3,1.0
82,51.165691,10.451526,Germany,9,0,0.0,1.0
98,22.396428,114.109497,Hong Kong,"1,755",299,17.0,1.0


In [28]:
countries_loc3['sars']  = countries_loc3['sars'].fillna(0)

In [30]:
dates = pd.date_range("2020-01-22", "2020-02-22")

all_data = dict()
for date in dates:
    print(date)
    confirmed_countries = covid4[covid4['ObservationDate'] <= date]['Country/Region']
    
    confirmed_loc = countries_loc3[countries_loc3['Country/Region'].isin(confirmed_countries)]
    confirmed_loc = confirmed_loc[['longitude', 'latitude', 'Country/Region']]

    data = countries_loc3.copy() ##
    next_confirmed = covid4[covid4['ObservationDate'] == date + pd.Timedelta(1,'D')]['Country/Region']
    
    if next_confirmed.shape[0] == 0:
        continue
    
    data['y'] = 0
    data.loc[data['Country/Region'].isin(next_confirmed), 'y'] = 1
    data.loc[data['Country/Region'].isin(confirmed_countries), 'y'] = 1 ##
    
    data['confirmed_past'] = 0 ##
    data.loc[data['Country/Region'].isin(confirmed_countries), 'confirmed_past'] = 1  ##
    data['date'] = date
    
    features = []
    for lon, lat, country in data[['longitude', 'latitude','Country/Region']].values:
        features.append(calc_confirmed_radius(lon, lat, country, confirmed_loc))
    
    features = pd.DataFrame(features)
    data = data.merge(features, how='left')
    data['Cases'] = data['Cases'].str.replace(',', '').astype(float).fillna(0)
    #print(data.head())
    #data['Deaths'] = data['Deaths'].str.replace(',', '').astype(float).fillna(0)
    
    all_data[date] = data
    print(data.shape[0], '\n')

2020-01-22 00:00:00
249 

2020-01-23 00:00:00
249 

2020-01-24 00:00:00
249 

2020-01-25 00:00:00
249 

2020-01-26 00:00:00
249 

2020-01-27 00:00:00
249 

2020-01-28 00:00:00
249 

2020-01-29 00:00:00
249 

2020-01-30 00:00:00
249 

2020-01-31 00:00:00
249 

2020-02-01 00:00:00
2020-02-02 00:00:00
2020-02-03 00:00:00
249 

2020-02-04 00:00:00
2020-02-05 00:00:00
2020-02-06 00:00:00
2020-02-07 00:00:00
2020-02-08 00:00:00
2020-02-09 00:00:00
2020-02-10 00:00:00
2020-02-11 00:00:00
2020-02-12 00:00:00
2020-02-13 00:00:00
249 

2020-02-14 00:00:00
2020-02-15 00:00:00
2020-02-16 00:00:00
2020-02-17 00:00:00
2020-02-18 00:00:00
249 

2020-02-19 00:00:00
2020-02-20 00:00:00
249 

2020-02-21 00:00:00
2020-02-22 00:00:00


# - O que é mais importante? Precision ou Recall?
- Add feature e baseline SARS (pra ser honesto)

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

features = ['avg_dist_to_confirmed', 'confirmed_1k', 'confirmed_2k', 'confirmed_3k', 'min_dist_to_confirmed', 'sars']#, 'Cases']
sorted_date = sorted(all_data.keys())


correct = {'Total': 0,'Baseline': 0, 'ML': 0, 'Rank Avg': 0, 'Baseline SARS': 0}
for i in range(len(sorted_date)-1) :

    
    date = sorted_date[i]
    next_date = sorted_date[i+1]
    
    Xtrain, ytrain = all_data[date][features], all_data[date]['y']
    Xval, yval = all_data[next_date][features], all_data[next_date]['y']
    Xval, yval = Xval[all_data[next_date]['confirmed_past'] == 0], yval[all_data[next_date]['confirmed_past'] == 0] ##
    
    pipe = make_pipeline(StandardScaler(), LogisticRegression(random_state=0, C=0.2, class_weight='balanced'))
    #pipe = DecisionTreeClassifier(max_depth=3,class_weight='balanced', ccp_alpha=0.0)
    #pipe = ExtraTreesClassifier(n_estimators=1000, n_jobs=6, max_depth=3, class_weight='balanced', random_state=0)
    #pipe = RandomForestClassifier(n_estimators=100, n_jobs=6, max_depth=3, class_weight='balanced', random_state=0)
    
    
    pipe.fit(Xtrain, ytrain)
    p = pipe.predict_proba(Xval)[:,1]
    
    Xval['p'] = p
    Xval['y'] = yval
    Xval['country'] = all_data[next_date]['Country/Region']
    
    top_n = 50
    baseline = Xval.sort_values("avg_dist_to_confirmed").head(top_n)['y'].sum()# / yval.sum()
    baseline_sars = Xval.sort_values("sars", ascending=False).head(top_n)['y'].sum()
    lr_top_20 = Xval.sort_values("p", ascending=False).head(top_n)['y'].sum()# / yval.sum()
    
    Xval['rank_avg'] = 0.9*Xval['avg_dist_to_confirmed'].rank() + 0.1*Xval['p'].rank(ascending=False)
    avg_in_top_20 = Xval.sort_values("rank_avg", ascending=True).head(top_n)['y'].sum() #/ yval.sum()
    
    correct['Total'] += yval.sum()
    correct['Baseline'] += baseline
    correct['ML'] += lr_top_20
    correct['Rank Avg'] += avg_in_top_20
    correct['Baseline SARS'] += baseline_sars
    
    str_result = "Prediction date: {}\nPositive in train: {}\nConfirmed next date: {}\nBaseline: {}\nML: {}\nRank avg top 20: {}\nBaseline SARS: {}\n".format( 
        date, ytrain.sum(), yval.sum(), baseline, lr_top_20, avg_in_top_20, baseline_sars)
    print(str_result)
    
### QUEM GANHA?

Prediction date: 2020-01-22 00:00:00
Positive in train: 10
Confirmed next date: 1
Baseline: 0
ML: 1
Rank avg top 20: 0
Baseline SARS: 1

Prediction date: 2020-01-23 00:00:00
Positive in train: 11
Confirmed next date: 3
Baseline: 2
ML: 2
Rank avg top 20: 2
Baseline SARS: 2

Prediction date: 2020-01-24 00:00:00
Positive in train: 14
Confirmed next date: 1
Baseline: 0
ML: 1
Rank avg top 20: 0
Baseline SARS: 1

Prediction date: 2020-01-25 00:00:00
Positive in train: 15
Confirmed next date: 3
Baseline: 2
ML: 2
Rank avg top 20: 2
Baseline SARS: 0

Prediction date: 2020-01-26 00:00:00
Positive in train: 18
Confirmed next date: 1
Baseline: 0
ML: 1
Rank avg top 20: 0
Baseline SARS: 1

Prediction date: 2020-01-27 00:00:00
Positive in train: 19
Confirmed next date: 2
Baseline: 2
ML: 0
Rank avg top 20: 2
Baseline SARS: 0

Prediction date: 2020-01-28 00:00:00
Positive in train: 21
Confirmed next date: 2
Baseline: 2
ML: 2
Rank avg top 20: 2
Baseline SARS: 2

Prediction date: 2020-01-29 00:00:00
Posi

In [32]:
correct

{'Total': 23, 'Baseline': 12, 'ML': 18, 'Rank Avg': 13, 'Baseline SARS': 12}

18/23 - recall @ 50 (~ 20%)

In [56]:
18/23

0.782608695652174

In [33]:
44/Xval.shape[0]

0.2018348623853211

In [34]:
#http://www.cidrap.umn.edu/news-perspective/2020/02/who-notes-covid-19-pandemic-potential-5-more-mideast-nations-affected
#24/02 - The five Middle Eastern countries reporting their first cases today include Afghanistan (83) , Bahrain (12), Kuwait (2), Iraq (40), and Oman (15)
#NOW: #SouthKorea confirms a new death due to #coronavirus, a 35-year-old man from Mongolia, making it the first death of a foreigner in the country.
# 25/02 - Áustria (19), Croácia (57), Espanha, Romênia (5) e Suíça (1) confirmam casos de Covid-19 
#Although there are no confirmed cases of #COVIDー19 in #Mongolia, the United States government is evacuating all #PeaceCorps volunteers in-country as a precaution to any potential infections from the disease.
#Indonesia
#Noruega confirmou https://www.cnbc.com/2020/02/26/coronavirus-latest-updates.html - 23º previsto
#Brasil (204)

In [35]:
Xval.sort_values("y", ascending=False).head(2)

,avg_dist_to_confirmed,confirmed_1k,confirmed_2k,confirmed_3k,min_dist_to_confirmed,sars,p,y,country,rank_avg
120,5739.674209,1,2,5,920.287873,0.0,0.396839,1,Lebanon,28.6
107,5855.885563,1,2,4,613.093748,0.0,0.450599,1,Israel,36.8


1. Suíça (confirmado)
2. Kuwait (confirmado)
3. Irlanda
4. Mongolia
5. Romênia (confirmado)
6. Indonésia
7. Mônaco
8. Qatar
9. Jersey
10. San Marino
11. Andorra
12. Bahrain (confirmado)
13. Luxemburgo
14. Holanda
15. Oman (confirmado)

In [ ]:
#http://www.cidrap.umn.edu/news-perspective/2020/02/who-notes-covid-19-pandemic-potential-5-more-mideast-nations-affected
#24/02 - The five Middle Eastern countries reporting their first cases today include Afghanistan (83) , Bahrain (12), Kuwait (2), Iraq (40), and Oman (15)
#NOW: #SouthKorea confirms a new death due to #coronavirus, a 35-year-old man from Mongolia, making it the first death of a foreigner in the country.
# 25/02 - Áustria (19), Croácia (57), Espanha, Romênia (5) e Suíça (1) confirmam casos de Covid-19 
#Although there are no confirmed cases of #COVIDー19 in #Mongolia, the United States government is evacuating all #PeaceCorps volunteers in-country as a precaution to any potential infections from the disease.
#Indonesia
#Noruega confirmou https://www.cnbc.com/2020/02/26/coronavirus-latest-updates.html - 23º previsto
#Brasil (204)

In [47]:
Xval2 = Xval.sort_values("p", ascending=False).reset_index(drop=True)
Xval2.head(50)

,avg_dist_to_confirmed,confirmed_1k,confirmed_2k,confirmed_3k,min_dist_to_confirmed,sars,p,y,country,rank_avg
0,6251.716425,4,7,9,464.652689,1.0,0.956073,0,Switzerland,55.0
1,5524.852675,2,3,3,686.055455,1.0,0.947285,0,Kuwait,13.7
2,6765.938528,2,7,8,371.652332,1.0,0.886829,0,Ireland,68.7
3,5143.128327,0,2,7,1223.581539,1.0,0.867731,0,Mongolia,10.3
4,5771.190404,0,6,10,1090.057321,1.0,0.724924,0,Romania,26.6
5,6767.202901,0,5,9,1148.119163,1.0,0.714771,0,Indonesia,69.9
6,6382.518237,5,7,9,467.958096,0.0,0.660057,0,Monaco,56.5
7,5546.185986,2,2,4,344.761763,0.0,0.643325,0,Qatar,17.0
8,6618.265326,5,7,8,464.551343,0.0,0.642998,0,Jersey,63.0
9,6185.517639,4,7,9,230.393457,0.0,0.606893,0,San Marino,50.5


In [50]:
Xval2[Xval2['country'] == 'Brazil']

,avg_dist_to_confirmed,confirmed_1k,confirmed_2k,confirmed_3k,min_dist_to_confirmed,sars,p,y,country,rank_avg
203,13470.202578,0,0,0,5659.930649,0.0,0.001609,0,Brazil,207.6


# Como melhorar?
- Alguém com conhecimento de epidemiologia
- ML brilhou depois do "óbvio" das baselines - Top 40, 50
    - O ranking é preditivo em geral, mas tem muito ruído
- Prever/avaliar quais países estão em risco para a próxima semana
    - Mais dados de treino
    - Menos ruído
- Treinar com todos os países confirmados vs apenas confirmados no "dia" anterior
- Por que é difícil?
    - Depende dos métodos de descoberta (Itália mede diferente de outros países da Europa, China parou de fazer apenas testes de laboratório)
    
- Países mais pobres
    - Em tese não terão a mesma capacidade de detecção dos países ricos
    - Mas também podem não ter tanta gente cruzando a fronteira
    
- Mais features:
    - Dados econômicos e sociais, saúde pública
    - Dados de parcerias comerciais (países que recebem muitos viajantes de países com casos confirmados)
    - Quantos confirmados em países a 1k Km, 2k Km...
    - Usar com dados da SARS (e outras doenças de propagação rápida)

- Olhar mais países do top N (30,50,100)
    - Desconsiderar previsões para países muito pequenos (Mônaco, San Marino, Andorra, Luxemburgo, Jersey)
    
- Plot do mundo (interativo?)

   

In [ ]:
Xval